In [ ]:
# Import libraries

import matplotlib.pyplot as plt
import pandas as pd
import warnings
from econml.sklearn_extensions.linear_model import WeightedLassoCV
import sklearn.model_selection as sms
import sklearn.linear_model as slm
import sklearn.preprocessing as skp
import sklearn.metrics as sme
import sklearn.feature_selection as skf
import sklearn.ensemble as ske
import sklearn.utils as sku
import sklearn.decomposition as skd
import sklearn.neural_network as skn
from celer import GroupLassoCV
from sklearnex import patch_sklearn, config_context
from sklearn.cluster import DBSCAN
import numpy as np
import scipy.stats as stats
from IPython.display import HTML
import util_arch as util
from scipy.spatial import cKDTree
import nibabel as nib
import os
import pickle
import torch
from torch import nn
from mlp_feats import MLP
from mlp_feats import train_model, model_cv
import loss_landscapes
import loss_landscapes.metrics
import copy
from torchviz import make_dot
from sklearn.ensemble import GradientBoostingRegressor
patch_sklearn()

In [ ]:
HTML('''
<style>
.jupyter-matplotlib {
    background-color: #000;
}

.widget-label, .jupyter-matplotlib-header{
    color: #fff;
}

.jupyter-button {
    background-color: #333;
    color: #fff;
}
</style>
''')

In [ ]:
# Maybe y should also be scaled? 
# Perhaps a transform would be more effective, or scaling implemented with consistent cross-validation
# Different scaling methods? 
#     This seems most important for noise-sensitive models like LARS. All other use StandardScaler()
# Transformers?
# Model-specific scaling methods?
#     Yes, see above
# Common cross-validation function ✓
#     Use built-in functions wherever possible and `utils.gridsearch_pickparams()` elsewhere
# Quantile loss
# RANSAC
# Data augmentation? (Mixup)
# Data generation? (SMOGN)
# Combine CHH dataset ✓
# Implement CV and test ✓
# Print selected features ✓
# Make magnitude templates
# Sample weights ✓
# Look at segmentations by error ✓ (Appears to have most difference in red nucleus, which includes surrounding (white?) matter for underperforming cases)
# Extract features from current (1:6) eroded ROIs ✓
# Test features from eroded ROIs
# Extract features from all ROIs
# Plot segmentation variance against error for each case across all ROIs ✓
# Why does excluding the subthalamic nucleus increase the correlation (r=0.5 -> r=0.6)?
# Best performance with all ROIs: cvn=6, k=1800
# Best performance with ROIs 0:4, excluding STN: cvn=6, k=1800
# Should the pre-operative UPDRS be appended once or to each ROI? ✓
# Plot histogram of features for successful and unsuccessful predictions ✓
# Check which cases are unilateral, such as 29, 44, 45, 46
# Compile bad case list: 2, 58, 59, 64, 68, 72, 75, 77, 78, 81, 85, 89, 90
# How many neurologists overall have been involved in the UPDRS-III scoring?

In [ ]:
# Get case IDs
case_list = open('/home/ali/RadDBS-QSM/data/docs/cases_90','r')
lines = case_list.read()
lists = np.loadtxt(case_list.name,comments="#", delimiter=",",unpack=False,dtype=str)
case_id = []
for lines in lists:     
    case_id.append(lines[-9:-7])

# Load scores
file_dir = '/home/ali/RadDBS-QSM/data/docs/QSM anonymus- 6.22.2023-1528.csv'
motor_df = util.filter_scores(file_dir,'pre-dbs updrs','stim','CORNELL ID')
# Find cases with all required scores
subs,pre_imp,post_imp,pre_updrs_off = util.get_full_cases(motor_df,
                                                          'CORNELL ID',
                                                          'OFF (pre-dbs updrs)',
                                                          'ON (pre-dbs updrs)',
                                                          'OFF meds ON stim 6mo')
# Load extracted features
npy_dir = '/home/ali/RadDBS-QSM/data/npy/'
phi_dir = '/home/ali/RadDBS-QSM/data/phi/phi/'
roi_path = '/data/Ali/atlas/mcgill_pd_atlas/PD25-subcortical-labels.csv'
n_rois = 6
all_rois = False
Phi_all, X_all, R_all, K_all, ID_all = util.load_featstruct(phi_dir,npy_dir+'X/',npy_dir+'R/',npy_dir+'K/',n_rois,1595,all_rois)

ids = np.asarray(ID_all).astype(int)
# Find overlap between scored subjects and feature extraction cases
c_cases = np.intersect1d(np.asarray(case_id).astype(int),np.asarray(subs).astype(int))
# Complete case indices with respect to feature matrix
c_cases_idx = np.in1d(ids,c_cases)
X_all_c = X_all[c_cases_idx,0:4,:]
K_all_c = K_all[c_cases_idx,0:4,:]
R_all_c = R_all[c_cases_idx,0:4,:]
print(np.unique(R_all_c))
# Re-index the scored subjects with respect to complete cases
s_cases_idx = np.in1d(subs,ids[c_cases_idx])
subsc = subs[s_cases_idx]
pre_imp = pre_imp[s_cases_idx]
post_imp = post_imp[s_cases_idx]
pre_updrs_off = pre_updrs_off[s_cases_idx]
per_change = post_imp
# Reshape keys and ROIs
if all_rois == True:
    K_all_cu = np.empty((K_all_c.shape[0],K_all_c.shape[1],K_all_c.shape[2]+1),dtype=object)
    K_all_cu[:,:,:-1] = K_all_c
    K_all_cu[:,:,-1] = 'pre_updrs'
    K = K_all_cu.reshape((K_all_cu.shape[0],K_all_cu.shape[1]*K_all_cu.shape[2]))[0]
    R = R_all_c.reshape((R_all_c.shape[0],R_all_c.shape[1]*R_all_c.shape[2]))
else:
    K = K_all_c.reshape((K_all_c.shape[0],K_all_c.shape[1]*K_all_c.shape[2]))[0]
    K = np.append(K,['pre_updrs'],0)
    R = R_all_c.reshape((R_all_c.shape[0],R_all_c.shape[1]*R_all_c.shape[2]))


In [ ]:
# qsms = util.full_path('/home/ali/RadDBS-QSM/data/nii/qsm')
# segs = util.full_path('/home/ali/RadDBS-QSM/data/nii/qsm')
# chi = []
# for j in np.arange(len(qsms)):
#     data = nib.load(qsms[j]).get_fdata()
#     if int(qsms[j][-9:-7]) == int(segs[j][-9:-7]):
#         try:
#             mask = nib.load(segs[j]).get_fdata()
#             img = util.pad_to((data[:,:,~(mask==0).all((0,1))])[192:320,192:320,:],128,128,108)
#             chi.append(img)
#             print('Loading',qsms[j],'of shape',str(img.shape))
#         except:
#             print('Skipping',qsms[j])
#             subsc = np.delete(subsc,j)
#             per_change = np.delete(per_change,j)
#             X_all_c = np.delete(X_all_c,j,axis=0)
#             pre_updrs_off = np.delete(pre_updrs_off,j)

In [ ]:
num_epochs = int(1e4)
scoring = 'r2'
results_bls = np.zeros_like(per_change)
results_ls = np.zeros_like(per_change)
results_gls = np.zeros_like(per_change)
curves = np.zeros((2,len(per_change),num_epochs))
gerror = np.zeros_like(per_change)
alphas = np.logspace(-4,-2,100)
Ks = []
Kstg = []
w = []
wg = []
visualize_loss = 1
num_neighbors = 8

In [ ]:
for j in np.arange(len(subsc)):
    test_id = subsc[j]
    test_index = subsc == test_id
    train_index = subsc != test_id
    X_train = X_all_c[train_index,:,:]
    X_test = X_all_c[test_index,:,:]
    y_train = per_change[train_index]
    y_test = per_change[test_index]
    # Cross validation
    cvn = 6
    X0_ss0,scaler_ss,X_test_ss0 = util.model_scale(skp.StandardScaler(),
                                                X_train,train_index,X_test,test_index,pre_updrs_off,False,False,False)
    with np.errstate(divide='ignore', invalid='ignore'):
      # Feature selection
      sel = skf.SelectKBest(skf.r_regression,k=1800)
      X0_ss = sel.fit_transform(X0_ss0,y_train)
      X_test_ss = sel.transform(X_test_ss0)
      #Ks.append(sel.transform(K.reshape(1, -1)))
      y_n = cKDTree(X0_ss).query(X_test_ss, k=1)[1]
    
    # LASSO
    lasso = slm.LassoCV(max_iter=1e4,cv=cvn,n_jobs=-1,alphas=alphas)
    est_ls = lasso.fit(X0_ss,y_train)
    results_ls[j] = est_ls.predict(X_test_ss)
    w.append(est_ls.coef_)
    
    # Gradient boosting
    # import xgboost as xgb
    # gsc = sms.GridSearchCV(
    #         estimator=xgb.XGBRegressor(n_jobs=1),
    #         param_grid={"learning_rate": (0.05, 0.10, 0.15),
    #                     "max_depth": [1,2,3,4,5],
    #                     "n_estimators": [10, 100],
    #                     "min_child_weight": [1, 3, 5, 7],
    #                     "gamma":[ 0.0, 0.1, 0.2],
    #                     "colsample_bytree":[ 0.3, 0.4],
    #                     "alpha":[1e-4, 1e-3, 1e-2, 1e-1]},
    #         cv=2, scoring='r2', n_jobs=-1)
    # est_gr = gsc.fit(X0_ss0, y_train)
    # results_bls[j] = est_gr.predict(X_test_ss0)
    # print(gsc.best_params_)
    # print(gsc.best_score_)

    print('Lasso predicts',str(np.round(results_ls[j],3)),
          #'and gradient boost predicts',str(np.round(results_bls[j],2)),
          'for case',str(int(subsc[j])),'with',str(np.round(per_change[j],2)),
          'with regularization',str(np.round(est_ls.alpha_,4)))#,'and',
          #str(np.round(est_gr.best_params_['alpha'],2)),
          #'and learning rate',str(est_gr.best_params_['learning_rate']))
    # MLP
    encoder = MLP()#in_size=X0_ss.shape[1],activation=nn.Softplus())
    model_initial = copy.deepcopy(encoder)
    # yt, encoder, X_trained, y_trained, X_val, y_val, train_curve, val_curve = train_model(X_all=X0_ss,
    #                  y_all=y_train,
    #                  model=encoder,
    #                  X_test=X_test_ss,
    #                  lr=1e-4,
    #                  lr_decay=500,
    #                  max_step=1,
    #                  alpha=est_ls.alpha_,
    #                  reg_type='l1',
    #                  num_epochs=num_epochs,
    #                  batch_size=X0_ss.shape[0]-num_neighbors,
    #                  case_id=str(int(subsc[j])),
    #                  num_neighbors=0,
    #                  random_val=True,
    #                  verbose=True,
    #                  save_state=False)
    # curves[0,j,:] = np.squeeze(train_curve)
    # curves[1,j,:] = np.squeeze(val_curve)
    yt, encoder, alphas, _ = model_cv(X_all=X0_ss,
                     y_all=y_train,
                     model=encoder,
                     X_test=X_test_ss,
                     lrs=np.asarray([1e-4]),
                     lr_decay=None,
                     alphas=np.logspace(-4,1,100),
                     reg_type='l1',
                     num_epochs=100,
                     batch_size=X0_ss.shape[0],
                     case_id=str(int(subsc[j])),
                     num_neighbors=0,
                     random_val=True,
                     verbose=False,
                     save_state=False,cvn=cvn)
    # if visualize_loss > 0:
    #   # Loss visualizations
    #   model_final = copy.deepcopy(encoder.cpu())
    #   criterion = torch.nn.MSELoss()
    #   metric = loss_landscapes.metrics.Loss(criterion, X_trained.cpu(), torch.unsqueeze(y_trained.cpu(),dim=1))
    #   steps = 40
 
    #   plt.plot(np.linspace(1,num_epochs,num_epochs),train_curve,'-',np.linspace(1,num_epochs,num_epochs),val_curve,'--')
    #   plt.title('Convergence')
    #   plt.xlabel('Epoch')
    #   plt.ylabel('Loss')
    #   plt.legend(['Training','Validation'])
    #   plt.show()
    #   plt.style.use('dark_background')
    # # if visualize_loss > 1:
    # #   # 1D loss
    # #   loss_data = loss_landscapes.linear_interpolation(model_initial, model_final, metric, steps ,deepcopy_model=True)
    # #   plt.plot([(1/steps)*i for i in range(steps)], loss_data)
    # #   plt.title('Linear Interpolation of Loss')
    # #   plt.xlabel(r'Interpolation Coefficient $\alpha$')
    # #   plt.ylabel(r'Loss $L(\theta(\alpha))$')
    # #   axes = plt.gca()
    # #   plt.show()
    # #   # 2D loss
    # #   loss_data_fin = loss_landscapes.random_plane(model_final, metric, 10, steps, normalization='filter', deepcopy_model=True)
    # #   plt.contour(loss_data_fin, levels=50)
    # #   plt.title('Loss Contours around Trained Model')
    # #   plt.show()
    # #   # 3D loss
    # #   fig = plt.figure()
    # #   ax = plt.axes(projection='3d')
    # #   X = np.array([[j for j in range(steps)] for i in range(steps)])
    # #   Y = np.array([[i for _ in range(steps)] for i in range(steps)])
    # #   ax.plot_surface(X, Y, loss_data_fin, rstride=1, cstride=1, cmap='viridis', edgecolor='none')
    # #   ax.set_title('Surface Plot of Loss Landscape')
    # #   plt.show()

    results_bls[j] = yt
    print('MLP predicts',str(np.round(yt.item(),3)),
            'for case',str(int(subsc[j])),'with',str(np.round(per_change[j],2)),
            'with regularization',str(alphas))

In [ ]:
results_bls[results_bls>1]=1

In [ ]:
util.eval_prediction(np.vstack((pre_imp,
                               results_ls,
                               results_bls,
                               )),
                               per_change,
                               ['LCT',
                                'Lasso',
                                'MLP'
                                ],(30,5))
plt.ylim([0,2])
plt.xlim([0,2])
plt.style.use('default')



In [ ]:
plt.plot(np.linspace(1,num_epochs,num_epochs),np.mean(curves[0,:,:],axis=0),np.linspace(1,num_epochs,num_epochs),np.mean(curves[1,:,:],axis=0))
plt.fill_between(np.linspace(1,num_epochs,num_epochs), np.mean(curves[0,:,:],axis=0)-np.std(curves[0,:,:],axis=0),np.mean(curves[0,:,:],axis=0)+np.std(curves[0,:,:],axis=0),
                 color='blue', alpha=0.2)
plt.fill_between(np.linspace(1,num_epochs,num_epochs), np.mean(curves[1,:,:],axis=0)-np.std(curves[1,:,:],axis=0),np.mean(curves[1,:,:],axis=0)+np.std(curves[1,:,:],axis=0),
                 color='orange', alpha=0.2)
plt.title('Convergence')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.ylim([-0,0.5])
plt.legend(['Training','Validation'])
plt.show()
plt.style.use('dark_background')